## CHECKING QUALITY OF RETRIEVED STRUCTURED EXPLANATION

In [1]:
from patterns.abstract import *
from patterns.entailment import *
from patterns.contradiction import *
from patterns.neutral import *
from functools import partial
import pandas as pd

In [2]:
se0 = StructuredExplanation('⊆', ('cat', 'animal'))
se1 = StructuredExplanation('∧', (StructuredExplanation('⊆', ('cat', 'animal')), StructuredExplanation('→', ('car', 'vehicle'), negated=True)))
se2 = StructuredExplanation('∧', (StructuredExplanation('o', ('car', 'vehicle')), StructuredExplanation('⊆', ('cat', 'animal'))))

print(se1)

cat ⊆ animal ∧ ¬(car → vehicle)


In [3]:
eval_data = pd.read_csv("data/cleaned_assigned_samples_training.csv")
gold_label = pd.read_csv("data/assigned_samples_training.csv")

In [4]:
def to_strEx(text: str) -> StructuredExplanation: 
    clauses = text.split('∧')
    strex = []
    for clause in clauses: 
        rel = re.findall("[↔|→|⇒|⊆|⊈|⊕|⊉]", clause)
        if rel:
            try:
                preds = tuple(s.strip(' .,!?¬()') for s in clause.split(rel[0]))
                strex.append(StructuredExplanation(rel[0], preds, negated=bool(re.findall("¬", clause))))
            except:
                print("error")
    return AbstractPattern.concatenate_explanations(strex)

In [5]:
to_strEx("¬(car → vehicle) ∧ cat ⊆ animal") == se1

True

In [24]:
def run_eval(eval_data, label_data, patterns, label, highlights=True):

    get_highlights = lambda n, r: ast.literal_eval(r[f'Sentence1_Highlighted_Ordered_{n}']) + ast.literal_eval(r[f'Sentence2_Highlighted_Ordered_{n}'])
    apply_patterns = lambda n, x: [se for se in [pattern(nlp(x[f'Explanation_{n}']), get_highlights(n, x) if highlights else []) for pattern in patterns] if se.relationship != '']
    concat = lambda x: AbstractPattern.concatenate_explanations(x)

    results = eval_data[eval_data["gold_label"] == label].copy()

    results['Explanation_1_Result'] = results.apply(partial(apply_patterns, 1), axis=1).apply(concat)

    output = results[[
        'Explanation_1', 'Explanation_1_Result'
    ]]

    output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)

    output.reset_index(drop=True, inplace=True)
    return output

#### Entailment

In [25]:
patterns = [RephrasingPattern(), ImplicationPattern(), EquivalencePattern(), IfThenPattern(), ClassificationPattern()]

In [26]:
ent_output = run_eval(eval_data, gold_label, patterns, "entailment", True)
ent_found = ent_output[ent_output['gold label'].apply(bool)]
ent_found['correct'] = ent_found['Explanation_1_Result'] == ent_found['gold label']

recall = sum(ent_found['correct'])/len(ent_found)
precision = sum(ent_found['correct'])/sum(ent_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(ent_found)

manual recall: 23.53%
manual precision: 57.14%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/1140757733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ent_found['correct'] = ent_found['Explanation_1_Result'] == ent_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
3,Going over jump is way to rephrase jumping dir...,is way to rephrase ↔ jumping dirt ramps,going over jump ↔ jumping dirt ramps,False
5,Riding a wave means one is outside.,,riding a wave → one is outside,False
6,Digs into the sand is a way of saying playing ...,is a way of saying ↔ playing in the sand is a ...,digs into the sand ↔ playing in the sand,False
8,"Men is plural, which would mean a group. Fris...",frisbee ⊆ game,frisbee ⊆ game,True
9,A boy playing with a camera means that he has ...,,a boy playing with a camera → he has a camera,False
12,A deck outside is outdoors.,,a deck outside ⊆ outdoors,False
14,If there is a rag tag of the blue bus then the...,rag tag of the blue bus ⇒ there is a blue bus ...,rag tag of the blue bus ⇒ there is a blue bus ...,True
19,Archery implies shooting arrows.,archery → shooting arrows,archery → shooting arrows,True
20,Jeans are usually made from denim.,,jeans ⊆ denim,False
21,A minivan would usually be found outside.,,minivan → outside,False


#### Contradiction

In [28]:
patterns = [NotRephrasingPattern(), NotImplicationPattern(), NotEquivalencePattern(), XORPattern(), IfThenPattern(), NotClassificationPattern(), CannotBePattern()]

In [29]:
con_output = run_eval(eval_data, gold_label, patterns, "contradiction", True)
con_found = con_output[con_output['gold label'].apply(bool)]
con_found['correct'] = con_found['Explanation_1_Result'] == con_found['gold label']

recall = sum(con_found['correct'])/len(con_found)
precision = sum(con_found['correct'])/sum(con_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(con_found)

manual recall: 38.10%
manual precision: 66.67%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/213622875.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_found['correct'] = con_found['Explanation_1_Result'] == con_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
1,If walking you cant drive at the same time and...,,walking ⇒ cant drive at the same time.\r\nfocu...,False
4,The worker is either looking for a tool or a p...,tool ⊕ paper,tool ⊕ paper,True
5,Its can either be a baseball or soccer ball no...,baseball ⊕ soccer,baseball ⊕ soccer,True
6,The man is not a dog.,man ⊈ dog,man ⊈ dog,True
7,One cannot be demonstrating and have a peacefu...,demonstrating ⊕ peaceful conversation,demonstrating ⊕ have a peaceful conversation,False
8,The girls can either be under a walkway or at ...,under a walkway ⊕ at a restaurant,under a walkway ⊕ at a restaurant,True
9,Riding bikes is not the same as playing video ...,,riding bikes not ↔ playing video games,False
10,A man can either only playing tennis or painting.,only playing tennis ⊕ painting,only playing tennis ⊕ painting,True
13,A man wouldn't usually be lying on his stomach...,,a man ⊕ a woman,False
16,A woman is not a boy.,woman ⊈ boy,woman ⊈ boy,True


#### Neutral

In [31]:
patterns = [NeutralImplicationPattern(), NotAllPattern()]
nn_output = run_eval(eval_data, gold_label, patterns, "neutral", True)
nn_found = nn_output[nn_output['gold label'].apply(bool)]
nn_found['correct'] = nn_found['Explanation_1_Result'] == nn_found['gold label']

recall = sum(nn_found['correct'])/len(nn_found)
precision = sum(nn_found['correct'])/sum(nn_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(nn_found)

manual recall: 9.52%
manual precision: 25.00%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/2523178807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nn_found['correct'] = nn_found['Explanation_1_Result'] == nn_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
0,Participating does not imply learning.,¬(participating → learning),¬(participating → learning),True
1,A man looking at something does not mean is he...,¬(a man looking at something → man looking at ...,¬(a man looking at something → he is shopping ...,False
2,People climbing does not imply they are a team.,¬(people climbing → team),¬(people climbing → they are a team),False
3,"Standing beside a ladder, wagon, and bales of ...",,"¬(standing beside a ladder, wagon, and bales o...",False
5,Not all workers placing tiles on walls are mak...,workers placing tiles on walls ⊉ bathroom,workers placing tiles on walls ⊉ bathroom,True
6,Man and woman standing in the desert with a ho...,,¬(man and woman standing in the desert with a ...,False
8,An older woman shopping for produce doesn't me...,,¬(an older woman shopping for produce → she is...,False
11,Standing outdoors does not necessarily mean ta...,,¬(standing outdoors → talking),False
12,That the woman has a shaved head does not nece...,,¬(that the woman has been shaved head → she is...,False
13,Just because the women are drinking from green...,,¬(they sit → they are sitting on pillows),False


## Not Using highlighted words

### Entailment

In [32]:
patterns = [RephrasingPattern(), ImplicationPattern(), EquivalencePattern(), IfThenPattern(), ClassificationPattern()]

In [33]:
ent_output = run_eval(eval_data, gold_label, patterns, "entailment", False)
ent_found = ent_output[ent_output['gold label'].apply(bool)]
ent_found['correct'] = ent_found['Explanation_1_Result'] == ent_found['gold label']

recall = sum(ent_found['correct'])/len(ent_found)
precision = sum(ent_found['correct'])/sum(ent_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(ent_found)

manual recall: 5.88%
manual precision: 14.29%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/833224169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ent_found['correct'] = ent_found['Explanation_1_Result'] == ent_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
3,Going over jump is way to rephrase jumping dir...,is way to rephrase ↔ jumping dirt ramps is way...,going over jump ↔ jumping dirt ramps,False
5,Riding a wave means one is outside.,,riding a wave → one is outside,False
6,Digs into the sand is a way of saying playing ...,is a way of saying ↔ playing in the sand is a ...,digs into the sand ↔ playing in the sand,False
8,"Men is plural, which would mean a group. Fris...",Frisbee ⊆ game,frisbee ⊆ game,False
9,A boy playing with a camera means that he has ...,,a boy playing with a camera → he has a camera,False
12,A deck outside is outdoors.,,a deck outside ⊆ outdoors,False
14,If there is a rag tag of the blue bus then the...,there is a rag tag of the blue bus ⇒ there is ...,rag tag of the blue bus ⇒ there is a blue bus ...,False
19,Archery implies shooting arrows.,archery → shooting arrows,archery → shooting arrows,True
20,Jeans are usually made from denim.,,jeans ⊆ denim,False
21,A minivan would usually be found outside.,,minivan → outside,False


#### Contradiction

In [34]:
patterns = [NotRephrasingPattern(), NotImplicationPattern(), NotEquivalencePattern(), XORPattern(), IfThenPattern(), NotClassificationPattern(), CannotBePattern()]

In [35]:
con_output = run_eval(eval_data, gold_label, patterns, "contradiction", False)
con_found = con_output[con_output['gold label'].apply(bool)]
con_found['correct'] = con_found['Explanation_1_Result'] == con_found['gold label']

recall = sum(con_found['correct'])/len(con_found)
precision = sum(con_found['correct'])/sum(con_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(con_found)

manual recall: 9.52%
manual precision: 16.67%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/1474848362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con_found['correct'] = con_found['Explanation_1_Result'] == con_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
1,If walking you cant drive at the same time and...,,walking ⇒ cant drive at the same time.\r\nfocu...,False
4,The worker is either looking for a tool or a p...,looking for a tool ⊕ a paper,tool ⊕ paper,False
5,Its can either be a baseball or soccer ball no...,be a baseball ⊕ soccer ball not both,baseball ⊕ soccer,False
6,The man is not a dog.,The man ⊈ a dog,man ⊈ dog,False
7,One cannot be demonstrating and have a peacefu...,demonstrating ⊕ have a peaceful conversation s...,demonstrating ⊕ have a peaceful conversation,False
8,The girls can either be under a walkway or at ...,be under a walkway ⊕ at a restaurant,under a walkway ⊕ at a restaurant,False
9,Riding bikes is not the same as playing video ...,,riding bikes not ↔ playing video games,False
10,A man can either only playing tennis or painting.,only playing tennis ⊕ painting,only playing tennis ⊕ painting,True
13,A man wouldn't usually be lying on his stomach...,,a man ⊕ a woman,False
16,A woman is not a boy.,A woman ⊈ a boy,woman ⊈ boy,False


#### Neutral

In [36]:
patterns = [NeutralImplicationPattern(), NotAllPattern()]
nn_output = run_eval(eval_data, gold_label, patterns, "neutral", False)
nn_found = nn_output[nn_output['gold label'].apply(bool)]
nn_found['correct'] = nn_found['Explanation_1_Result'] == nn_found['gold label']

recall = sum(nn_found['correct'])/len(nn_found)
precision = sum(nn_found['correct'])/sum(nn_found['Explanation_1_Result'].apply(bool))
print(f"manual recall: {recall*100:.2f}%")
print(f"manual precision: {precision*100:.2f}%")

display(nn_found)

manual recall: 4.76%
manual precision: 12.50%


/tmp/ipykernel_103782/558741555.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['gold label'] = label_data['structured_explanation'].apply(str).apply(str.lower).apply(to_strEx)
/tmp/ipykernel_103782/1353684545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nn_found['correct'] = nn_found['Explanation_1_Result'] == nn_found['gold label']


,Explanation_1,Explanation_1_Result,gold label,correct
0,Participating does not imply learning.,¬(participating → learning),¬(participating → learning),True
1,A man looking at something does not mean is he...,¬(a man looking at something → man looking at ...,¬(a man looking at something → he is shopping ...,False
2,People climbing does not imply they are a team.,¬(people climbing → climbing they are a team),¬(people climbing → they are a team),False
3,"Standing beside a ladder, wagon, and bales of ...",,"¬(standing beside a ladder, wagon, and bales o...",False
5,Not all workers placing tiles on walls are mak...,workers placing tiles on walls ⊉ making a bath...,workers placing tiles on walls ⊉ bathroom,False
6,Man and woman standing in the desert with a ho...,,¬(man and woman standing in the desert with a ...,False
8,An older woman shopping for produce doesn't me...,,¬(an older woman shopping for produce → she is...,False
11,Standing outdoors does not necessarily mean ta...,,¬(standing outdoors → talking),False
12,That the woman has a shaved head does not nece...,,¬(that the woman has been shaved head → she is...,False
13,Just because the women are drinking from green...,,¬(they sit → they are sitting on pillows),False
